In [22]:
import matplotlib.pyplot as plt
import numpy as np
import math

f, U_aus = np.loadtxt('Verstaerker.txt', skiprows=1, usecols=(0,1), unpack =True)
D=1e-3     # Abschwächung
U_ein=0.21
g=U_aus/(U_ein*D)

In [29]:
from scipy.optimize import curve_fit
def fit_func(f, V, W1, W2, n1, n2):
    return V/(np.sqrt(1+1/(f/W1)**(2*n1))*np.sqrt(1+(f/W2)**(2*n2)))
p0=[1000,1000,50000,5,5]
popt,pcov=curve_fit(fit_func,f,g,p0)
print('Verstärkung: {v:.3e}'.format(v=popt[0]))

print('1sigma-Fehler der Verstärkung:{value:.2}'.format(value=math.sqrt(pcov[0,0])))

Verstärkung: 1.034e+03
1sigma-Fehler der Verstärkung:1.0


In [30]:
plt.loglog(f, g, linestyle='None', marker='.', label='Messdaten')
plt.loglog(f, fit_func(f, *popt), label='Fit')
plt.axis([3E2, 1.5e5,10,1.5E3])
plt.xlabel('Frequenz/Hz')
plt.ylabel('g(f)')
plt.title('Frequenzgang')
plt.legend(loc='best')
plt.savefig("243FrequenzgangFit.pdf",format="pdf",dpi=1000.)

In [31]:
import scipy.integrate as integrate
def fit_func_square(f,V,W1,W2,n1,n2):
    return fit_func(f,V,W1,W2,n1,n2)**2
B=integrate.quad(fit_func_square,f[1],f[-1],args=tuple(popt))
print('Das Integral betraegt:{value:.4e}'.format(value=B[0]))

Das Integral betraegt:4.9747e+10


In [34]:
# Messung der Rauschspannung als Funktion des ohmschen Widerstands
R = np.array([5e3,10e3,15e3,20e3,25e3,30e3])                           # Widerstände
fehler_R = R*0.005                                                     # Fehler der Widerstände 0.5%
U_aus = np.array([2.4071,3.1391,3.7273,4.2472,4.7139,5.1318])          # Ausgangspannung
fehler_U_aus = np.array([0.0011,0.0011,0.0014,0.0018,0.0018,0.0020])   # statistische Fehler
fehler_U_aus = np.sqrt((0.003*U_aus)**2 + fehler_U_aus**2)             # Zusammen mit die Messgenauigkeit des Voltmeters 0.3%
U_V=1.3919                                                             # Verstärkerrauschen
fehler_U_V=math.sqrt(0.00083**2+(U_V*0.003)**2)                        # Wie bei Ausgangspannung
D=U_aus**2-U_V**2                                                      # Rauschspannung des Widerstands
fehler_D=np.sqrt([(2*fehler_U_aus[i]*U_aus[i])**2 + (2*fehler_U_V*U_V)**2 for i in range(0,6)])

In [35]:
# Fit
def linear(x,c):
    return c*x
p02=[0.00081]
popt2,pcov2=curve_fit(linear,R,D,p02)

In [63]:
# Rechne k nach Gleichung (9)
c = popt2[0]
fehler_c = math.sqrt(pcov2[0])
T = 297.05                                        # T = 23.9°C
fehler_T = 0.1
k = c*1e-6 / (4*T*B[0])                           # Einheit von c muss von mV^2/ohm zu V^2/ohm umwandelt werden
syst_k = k * math.sqrt((fehler_T/T)**2 + 0.02**2) # systematische Fehler wegen Fehler von T und B, Fehler von B beträgt 2%
stat_k = k * (fehler_c/c)                         # statistische Fehler wegen Integralrechnung

plt.clf()
plt.errorbar(R,D,xerr=fehler_R,yerr=fehler_D,fmt=',',label='Messdaten')
plt.axis([0,32e3,0,25])
plt.plot(R, linear(R, *popt2), label='Fit')
plt.xlabel('Widerstand/ $\Omega$')
plt.ylabel('$(U_{aus}^2-U_V^2)$ / $mV^2$')
plt.title('Rauschspannung als Funktion von Widerstand')
plt.legend(loc='best')
plt.text(500, 19.5, 'Steigung: c = {v:.4e}'.format(v=popt2[0])+' $mV^2$/$\Omega$')   # Fehler von c sehr klein ~1e-11
plt.text(500, 2, 'k = (%.3f'%(k*1e23) + '$\pm$' +'%.3f'%(stat_k*1e23) + ' stat. $\pm$' + '%.3f'%(syst_k*1e23) + ' syst.) x$10^{-23}$ J/K')
plt.savefig("243Rauschspannung_Widerstand.pdf",format="pdf",dpi=1000.)

# überprüfe die Qualität des Anpassens
chisquare=np.sum(((linear(R,c)-D)**2/fehler_D**2))
dof=5                                             # 6 Widerstände, degree of freedem = 6 - 1
chisquare_red=chisquare/dof
print('chisquare = {value:.4}'.format(value=chisquare))
print('chisquare_red = {value:.3}'.format(value=chisquare_red))
# rechne die Wahrscheinlichkeit, dass wir bei einer Wiedelholungmessung ein chisquare-Wert erhalten, die >= diese Wert ist
from scipy.stats import chi2
prob=round(1-chi2.cdf(chisquare,dof),2)*100
print("Fitwahrscheinlichkeit = "+str(prob)+"%")   

chisquare = 37.76
chisquare_red = 7.55
Fitwahrscheinlichkeit = 0.0%


In [65]:
# Temperaturabhängigkeit
temp1 = np.array([51.3,102.9,153.3,205])
temp2 = np.array([51.4,103.1,153.5,205.1])
temp = np.array([(temp1[i] + temp2[i]) / 2 + 273.15  for i in range(0, 4)])
fehler_temp = np.array([max(abs(temp[i] -273.15 - temp1[i]), abs(temp[i] - 273.15 - temp2[i])) for i in range(0,4)])
Rvor = np.array([4.7967, 5.584, 6.343, 7.1073])     # in tausendOhm
Rnach = np.array([4.7979,5.588,6.3452, 7.109])
R2 = (Rvor + Rnach) / 2
fehler_R2 = np.array([max(abs(R2[i] - Rvor[i]), abs(R2[i] - Rnach[i])) for i in range(0,4)])
mu = np.array([2.4711,2.7536,3.0316,3.3226])
fehler_mu = np.array([0.001,0.001,0.0012,0.0012])
fehler_mu = np.sqrt((0.003*mu)**2 + fehler_mu**2) 
y = np.array([(mu[i]**2 - U_V**2)*1e-9 / R2[i] for i in range(0, 4)])
fehler_y = np.array([y[i] * math.sqrt((2*fehler_mu[i]/mu[i])**2 + (2*fehler_U_V/U_V)**2 + (fehler_R2[i]/R2[i])**2) for i in range(0, 4)])

# Fit a), Steigung als freie Parameter
p03=[1e-12]
popt3,pcov3=curve_fit(linear,temp,y,p03)
c = popt3[0]
fehler_c = math.sqrt(pcov3[0])
k = c / (4 * B[0])
print('die Steigung von Fit a) betraegt: c\' = {v:.4e}'.format(v = c) )
print('Fehler der Steigung : {v:.1e}'.format(v = fehler_c))
print('Da der relative Fehler von c\' sehr klein ist, für die Fehlerrechnung der Boltzmannkonstante benutzen wir nur den Fehler von B')
fehler_k = k * 0.02
print('k = ({v:.4}'.format(v = k*1e23), '±', '{v:.2})'.format(v =fehler_k*1e23), '×10^(-23) J/K')

# Fit b), Steigung und Temperaturnullpunkt als freie Parameter
def lin(x, c, b):
    return c*x + b
p04=[1e-12, 0]
popt4,pcov4=curve_fit(lin,temp,y,p04)
print('')
print('die Steigung von Fit b) betraegt: {v:.3e}'.format(v = popt4[0]))
print('Fehler der Steigung : {v:.1e}'.format(v = math.sqrt(pcov4[0][0])))
# absolute Nullpunkt
NullP = -popt4[1]/popt4[0]
fehler_NullP = -NullP * math.sqrt(pcov4[0][0]/popt4[0]**2)               # Fehler von den Abschnittpunkt mit y-Achse zu klein
print('der absolute Nullpunkt betraegt: -b/c = {v:.5}'.format(v = NullP))
print('Fehler der absolute Nullpunkt:  {v:.2}'.format(v = fehler_NullP))

plt.clf()
plt.errorbar(temp, y, xerr = fehler_temp, yerr = fehler_y, fmt='.',label='Messdaten')
x = np.linspace(0, 500, 500)
plt.plot(x, linear(x,c), label='Fit a)')
plt.plot(x, lin(x, popt4[0], popt4[1]), label='Fit b)')
plt.axis([0,550,0, 1.4e-9])
plt.xlabel('Temperatur  / K')
plt.ylabel('$(U_{aus}^2-U_V^2)$ / R   /   $V^2 / \Omega$')
plt.legend(loc='best')
plt.savefig("243Rauschspannung_Temperatur.pdf",format="pdf",dpi=1000.)

die Steigung von Fit a) betraegt: c' = 2.6804e-12
Fehler der Steigung : 1.9e-15
Da der relative Fehler von c' sehr klein ist, für die Fehlerrechnung der Boltzmannkonstante benutzen wir nur den Fehler von B
k = (1.347 ± 0.027) ×10^(-23) J/K

die Steigung von Fit b) betraegt: 2.673e-12
Fehler der Steigung : 1.6e-14
der absolute Nullpunkt betraegt: -b/c = -1.1065
Fehler der absolute Nullpunkt:  0.0067
